In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
def identity(a): return a
def rotCW(a): return np.fliplr(a.T)
def rotCCW(a): return np.flipud(a.T)

In [ ]:
moves = {
    "left": [identity, identity],
    "up": [rotCCW, rotCW],
    "right": [np.fliplr, np.fliplr],
    "down": [rotCW, rotCCW]
}

In [ ]:
def nextState(state, move):
    transform, invTransform = moves[move]
    nst = np.copy(transform(state))
    nst = nst[np.indices((4,4))[0], np.argsort(nst==0)]
    for ind in range(3):
        cmp = (nst[:, ind] == nst[:, ind+1]) * (nst[:, ind] > 0)
        nst[:, ind] += cmp
        nst[cmp, ind+1] = -1
    nst = nst[np.indices((4,4))[0], np.argsort(nst == -1)]
    nst[nst==-1] = 0
    return invTransform(nst)

In [ ]:
def perturb(state):
    zeros = np.argwhere(state == 0)
    if len(zeros) > 0:
        nst = np.copy(state)
        loc = np.random.randint(zeros.shape[0])
        row, col = zeros[loc, :]
        nst[row, col] = 1
        return nst
    else:
        return state

In [ ]:
def isDone(state):
    numZeros = np.sum(state == 0)
    if numZeros > 0:
        return False
    else:
        up = nextState(state, 'up')
        left = nextState(state, 'left')
        return np.all(up == left)

In [ ]:
def playToCompletion(strategy, nGames):
    initState = np.array([
        [1,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
    ])
    state = np.copy(initState)
    record = []
    for game in range(nGames):
        moveCount = 0
        movesTaken = []
        while not isDone(state):
            move = strategy(state)
            newState = nextState(state, move)
            if not np.all(newState == state):
                state = perturb(newState)
                moveCount += 1
                movesTaken += [move]
        record += [(moveCount, np.max(state))]
        state = np.copy(initState)
    return record

In [ ]:
def randomStrategy(choices):
    def strategy(state):
        return np.random.choice(choices)
    return strategy

In [ ]:
choiceStrategies = [
    ['down', 'left'] * 30 + ['right']*1 + ['up'],
    ['down', 'left'] * 30 + ['right']*3 + ['up'],
    ['down', 'left'] * 30 + ['right']*10 + ['up'],
    ['down', 'left'] * 30 + ['right']*30 + ['up'],
    ['down', 'left'] * 30 + ['right']*100 + ['up'],
]

result = [playToCompletion(randomStrategy(vec), 100) for vec in choiceStrategies]

In [ ]:
plt.figure(figsize=(16,5))
for ind, x in enumerate(result):
    plt.subplot(151+ind, ylim=(0,60))
    plt.hist([score for mvs, score in x])
plt.show()

In [ ]:
st = np.array([
    [
        [1, 0, 1, 0],
        [2, 2, 0, 0],
        [1, 0, 1, 0],
        [2, 2, 0, 0],
    ],
    [
        [1, 1, 1, 1],
        [2, 2, 1, 1],
        [1, 2, 1, 2],
        [2, 2, 1, 1],
    ],
    [
        [1, 2, 3, 2],
        [2, 3, 2, 1],
        [1, 2, 3, 2],
        [2, 3, 2, 1],
    ],
])

In [ ]:
def rotCWMulti(st): return np.rot90(st, axes=(1,2))
def rotCCWMulti(st): return np.rot90(st, axes=(2,1))
def flipMulti(st): return np.flip(st, axis=1)

movesMulti = {
    "up": [identity, identity],
    "left": [rotCCWMulti, rotCWMulti],
    "down": [flipMulti, flipMulti],
    "right": [rotCWMulti, rotCCWMulti]
}

def nextStateMulti(state, move):
    transform, invTransform = movesMulti[move]
    nst = np.copy(transform(state))
    nst = nst[np.indices(nst.shape)[0], np.argsort(nst==0, axis=1), np.indices(nst.shape)[2]]
    for ind in range(3):
        cmp = (nst[:, ind] == nst[:, ind+1]) * (nst[:, ind] > 0)
        nst[:, ind, :] += cmp
        nst[:, ind+1, :][cmp] = -1
    nst = nst[np.indices(nst.shape)[0], np.argsort(nst==-1, axis=1), np.indices(nst.shape)[2]]
    nst[nst==-1] = 0
    return invTransform(nst)

In [ ]:
def isFullMulti(state):
    nGames = state.shape[0]
    return np.sum(state.reshape(nGames, -1) == 0, axis=1) == 0

def isDoneMulti(state):
    nGames = state.shape[0]
    return isFullMulti(state) & np.all((nextStateMulti(state, 'left') == nextStateMulti(state, 'up')).reshape(nGames, -1),
                 axis=1)

def hasChanged(oldState, newState):
    nGames = oldState.shape[0]
    return np.invert(np.all((oldState == newState).reshape(nGames, -1), axis=1))

In [ ]:
def perturbMulti(state):
    notFull = np.invert(isFullMulti(state))
    tmp = state[notFull]
    for ind, game in enumerate(tmp):
        zeros = np.argwhere(game == 0)
        loc = np.random.randint(zeros.shape[0])
        row, col = zeros[loc, :]
        game[row, col] = 1
    state[notFull] = tmp

In [ ]:
def playToCompletionMulti(strategy, nGames, numTurns = 5):
    initState = np.array([[
        [1,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
    ]])
    state = np.repeat(initState, repeats=nGames, axis=0)
    complete = np.repeat(False, repeats=nGames)
    moveCount = np.zeros(nGames)
    for turn in range(numTurns):
        if np.all(complete):
            print(turn)
            break
        incomplete = np.invert(complete)
        
        liveState = state[incomplete]
        movesToMake = strategy(liveState)
        for m in ['up', 'left', 'down', 'right']:
            gamesToUpdate = movesToMake == m
            liveState[gamesToUpdate] = nextStateMulti(liveState[gamesToUpdate], m)
            
        didChange = hasChanged(state[incomplete], liveState)
        if np.any(didChange):
            movesMade = np.zeros(np.sum(incomplete))
            movesMade[didChange] = 1
            moveCount[incomplete] += movesMade
            changed = liveState[didChange]
            perturbMulti(changed)
            liveState[didChange] = changed
            state[incomplete] = liveState
            complete[incomplete] = isDoneMulti(liveState)
    return moveCount, complete, state

In [ ]:
def randomStrategyMulti(choices):
    def strategy(state):
        return np.random.choice(choices, size=state.shape[0])
    return strategy

In [ ]:
moveCounts, complete, finalStates = playToCompletionMulti(randomStrategyMulti(choiceStrategies[3]), 1000, numTurns=10000)

In [ ]:
plt.hist(np.max(finalStates.reshape(1000, -1), axis=1), bins=[0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5])
plt.show()

In [ ]:
finalStates[moveCounts>350]

In [ ]:
import tensorflow as tf
from operator import itemgetter

In [ ]:
tf.reset_default_graph()

In [ ]:
inputs = tf.placeholder(shape=(1,4,4,12), dtype=tf.float32)
conv1 = tf.layers.conv2d(
    inputs=inputs,
    filters=48,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu
)
conv1_flat = tf.reshape(conv1, [1, 4*4*48])
dense1 = tf.layers.dense(inputs=conv1_flat, units=96, activation=tf.nn.relu)
Qout = tf.layers.dense(inputs=dense1, units=4, activation=tf.nn.relu)

In [ ]:
nextQ = tf.placeholder(shape=[1,4], dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.00001)
updateModel = trainer.minimize(loss)

In [ ]:
def initState():
    return np.array([
        [1,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
    ])

def stateToNNInput(s):
    return np.eye(12)[s].reshape([1, 4, 4, 12])

moveNames = ['up','left','down','right'];

def selectMove(state, Q):
    allowedMoves = np.array([np.any(state != nextState(state, move)) for move in ['up', 'left', 'down', 'right']])
    return np.arange(0, 4)[allowedMoves][np.argmax(Q[allowedMoves])]

def randomAction(s):
    return selectMove(s, np.random.rand(4))

def nextStep(state, action):
    newState = nextState(state, moveNames[action])
    if np.all(newState == state):
        reward = -0.1
        return newState, reward, False, True
    else:
        newState = perturb(newState)
    
    if isDone(newState):
        reward = 2.0 ** np.max(newState)
        return newState, reward, True, False
    else:
        reward = 0.1 * np.sum(newState == 0)
        return newState, reward, False, False

In [ ]:
from collections import Counter
from IPython import display
import time

In [ ]:
y = 0.99
e = 0.1
num_episodes = 10000
jList1 = []
rList1 = []
topScoringS1 = None
topScore1 = 0.0
moveIndices = { name: ind for ind, name in enumerate(moveNames)}
strategy = randomStrategy([moveIndices[move] for move in choiceStrategies[0]])
for i in range(num_episodes):
    s = initState()
    rAll = 0
    d = False
    j = 0
    actions = []
    while j < 2000:
        j+=1
        a = strategy(s)
        if np.random.rand(1) < e: a = randomAction(s)
        s1,r,d,f = nextStep(s, a)
        actions.append("x" if f else moveNames[a])
        rAll += r
        if (i+1) % 200 == 0:
            display.clear_output(wait=True)
            print("{}".format(actions[-1]))
            print(s)
            print(r, rAll)
            time.sleep(0.1)
        s = s1
        if d == True:
            if rAll > 500:
                e = 1./((i/50) + 10)
            break
    if rAll > topScore1:
        topScoringS1 = s
        topScore1 = rAll
    print("Episode {} of {}: {}, {}".format(i, num_episodes, rAll, j), end='\r')
    jList1.append(j)
    rList1.append(rAll)
    if (i+1) % 50 == 0:
        display.clear_output(wait=True)
        plt.figure(figsize=(10,5))
        plt.subplot(121)
        plt.plot(jList1, [np.log2(r) for r in rList1], '.')
        plt.subplot(122)
        print(topScoringS1)
        plt.imshow(topScoringS1)
        plt.show()